# Modelowanie przy użyciu metody najmniejszych kwadratów

W pierwszym podejściu, modelujemy zadane wyjścia przy zastosowaniu metody najmniejszych kwadratów. Przetestowane zostaną modele liniowe oraz nieliniowe o różnych stopniach nielinowości.

### Zaimportuj potrzebne biblioteki

In [102]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(mpl.__version__))

pandas version: 1.0.1
numpy version: 1.18.1
matplotlib version: 3.2.0


### Zamień datę na sekundy od początku eksperymentu

In [103]:
def changeDateToSeconds(df):
    first = df["date"][0]
    df["date"] = df["date"].apply(lambda timestamp: (timestamp-first).seconds)
    return df

### Wczytaj dane

In [104]:
def readDataFromExcel(path, sheet):
    df = pd.read_excel(path, sheet_name=sheet)
    df["date"] = pd.to_datetime(df["date"])
    df = changeDateToSeconds(df)
    return df

### Wczytaj zbiór uczący i weryfikacyjny

In [114]:
df_learn = readDataFromExcel("./data/K-1_MI.xlsx", "d2")
df_verif = readDataFromExcel("./data/K-1_MI.xlsx", "d6")

### Zbiór uczący 

In [29]:
df_learn.head()

,date,FP05,LT1,LT2,LT3,LT4,TMA,TMB,TMC,TMD,...,PTWS,TW02,TW01,FW03,TW04,TW03,FW04,TTWT,PTWT,PPW
0,0,1068.9575,21.4107,11.6199,11.714,18.8643,43.8947,76.1302,75.8843,74.866,...,19.3311,539.8597,301.2129,9.4894,539.5955,290.8920,14.2998,177.7786,10.3202,3.5039
1,10,1068.9575,21.4107,11.6199,11.714,18.8643,43.8947,76.1302,75.8843,74.866,...,19.3311,539.8597,301.2129,9.4894,539.5955,290.8920,14.2998,177.7786,10.3202,3.5039
2,20,1068.9575,21.4107,11.6199,11.714,18.8643,43.8947,76.1302,75.8843,74.866,...,19.3147,539.8597,301.2129,9.4894,539.5955,292.1826,14.2998,177.7786,10.3202,3.4937
3,30,1068.9575,21.4107,11.6199,11.714,18.8643,43.8947,76.1302,75.8843,74.866,...,19.3298,539.8597,302.4265,9.4894,539.5955,292.1826,14.2998,177.7786,10.3202,3.4937
4,40,1068.9575,21.4107,11.6199,11.714,18.8643,43.8947,76.1302,75.8843,74.866,...,19.3298,539.8597,302.4265,9.4894,539.5955,292.1826,14.2998,177.7786,10.3202,3.4937


### Zbiór weryfikacyjny

In [30]:
df_verif.head()

,date,FP05,LT1,LT2,LT3,LT4,TMA,TMB,TMC,TMD,...,PTWS,TW02,TW01,FW03,TW04,TW03,FW04,TTWT,PTWT,PPW
0,0,1082.9224,6.18,-2.7184,11.714,5.1335,41.8797,74.6912,76.2002,74.9009,...,19.6512,539.3313,322.0861,6.1886,539.5955,322.4590,7.8027,176.6922,10.5121,3.4832
1,10,1082.9224,6.18,-2.7184,11.714,5.1335,41.8797,74.6912,76.2002,74.9009,...,19.6512,539.3313,322.0861,6.1886,539.5955,322.4590,7.8027,176.6922,10.5121,3.4832
2,20,1082.9224,6.18,-2.7184,11.714,5.1335,41.8797,74.6912,76.2002,74.9009,...,19.6512,539.3313,320.1805,6.1886,539.5955,319.8581,7.8027,176.6922,10.5121,3.4832
3,30,1082.9224,6.18,-2.7184,11.714,5.1335,41.8797,74.6912,76.2002,74.9009,...,19.6512,540.3870,317.8844,6.1886,539.5955,318.6641,7.8027,176.6922,10.5604,3.4938
4,40,1082.9224,6.18,-2.7184,11.714,5.1335,41.8797,74.6912,76.2002,74.9009,...,19.6512,540.3870,316.8332,0.0000,539.5955,317.4584,0.0000,176.6922,10.6015,3.4805


### MNK - model statyczny

In [115]:
u_learn = df_learn.drop(["LT01", "DP", "date"], axis=1).to_numpy()
y_learn = df_learn[["LT01", "DP"]].to_numpy()

u_verif = df_verif.drop(["LT01", "DP", "date"], axis=1).to_numpy()
y_verif = df_verif[["LT01", "DP"]].to_numpy()


### Weryfikacja modelu

In [116]:
reg = LinearRegression().fit(u_learn, y_learn)
y_model_learn = reg.predict(u_learn)
y_model_verif = reg.predict(u_verif)

print("Score learn: {}".format(r2_score(y_learn, y_model_learn)))
print("Score verif: {}".format(r2_score(y_verif, y_model_verif)))

Score learn: 0.68981181987722
Score verif: -6.578488912345968e+18
